In [1]:
# Base Libraries
from operator import index
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
# Transformation
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import power_transform
from sklearn.pipeline import Pipeline
# Feature Selection
import sklearn_relief as sr # type: ignore
# Models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [8]:
selectVA = 'arousal'
#selectVA = 'valence'

noFeatures = 10

In [9]:
np.set_printoptions(suppress=True)
subfolder = 'Dataset/'
featuresdf = pd.read_pickle(subfolder+'pickle/199_exported_features_valence_arousal2021.pkl')

X = np.array(featuresdf['features'].tolist()) # input
Y = np.array(featuresdf[selectVA].tolist())
y = Y.T

# remove some feature by hand
#print(len(X[0]))
#for i in range(139, 12, -1): # remove MEL spectrogram
#    X = np.delete(X, i, 1) # input
#print(len(X[0]))

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)

In [53]:
# Filter
r = sr.RReliefF(n_features = noFeatures)
my_transformed_matrix = r.fit_transform(X_train,y_train)



In [23]:
print(len(my_transformed_matrix))
print(len(X))

1261
1802


In [54]:
indexes = []
for value in my_transformed_matrix[0]:
    for idx, origVal in enumerate(X_train[0]):
        if(value == origVal):
            indexes.append(idx)
            break
indexes.sort()
print(len(indexes))
print()
print(indexes)


10

[52, 53, 54, 55, 56, 59, 65, 68, 69, 198]


In [55]:
features = np.array(featuresdf['features'].tolist())

selectedFeatures = []
for indexExample, example in enumerate(features):
    exampleFeatures=[]
    for index in indexes:
        exampleFeatures.append(example[index])
    selectedFeatures.append([exampleFeatures, y[indexExample]])

print(len(selectedFeatures))
print(len(selectedFeatures[0]))

featuresdfNew = pd.DataFrame(selectedFeatures, columns=['features',selectVA])
featuresdfNew.to_pickle(subfolder+'pickle/reliefF_'+str(noFeatures)+'_features_wo_MELspectrogram_'+selectVA+'.pkl')
#print(featuresdfNew)

1802
2


In [4]:
# Base Libraries
from operator import index
import numpy as np 
import pandas as pd
# Feature Selection
import sklearn_relief as sr # type: ignore
# Models
from sklearn.model_selection import train_test_split

from commonFunctions import featuresTranslation

np.set_printoptions(suppress=True)
subfolder = 'Dataset/'
featuresdf = pd.read_pickle(subfolder+'pickle/199_exported_features_valence_arousal2021.pkl')

X = np.array(featuresdf['features'].tolist()) # input

# remove some feature by hand
#print(len(X[0]))
#for i in range(139, 12, -1): # remove MEL spectrogram
#    X = np.delete(X, i, 1) # input
#print(len(X[0]))

VA = ['valence', 'arousal']

noFeatures = 5
while noFeatures > 0:
    for selectVA in VA:
        print()
        print(selectVA+" "+str(noFeatures))
        
        Y = np.array(featuresdf[selectVA].tolist())
        y = Y.T
        
        X_train, X_features, y_train, y_features = train_test_split(X,y, test_size = 0.2, random_state = 0)
        
        # RReliefF filter
        r = sr.RReliefF(n_features = noFeatures)
        my_transformed_matrix = r.fit_transform(X_features,y_features)
        
        indexes = []
        for value in my_transformed_matrix[0]:
            for idx, origVal in enumerate(X_features[0]):
                if(value == origVal):
                    indexes.append(idx)
                    break
        #print(indexes)
        print(featuresTranslation(indexes))
        
        #indexes.sort()
        #features = np.array(featuresdf['features'].tolist())
        #selectedFeatures = []
        #for indexExample, example in enumerate(features):
        #    exampleFeatures=[]
        #    for index in indexes:
        #        exampleFeatures.append(example[index])
        #    selectedFeatures.append([exampleFeatures, y[indexExample]])
        
        #featuresdfNew = pd.DataFrame(selectedFeatures, columns=['features',selectVA])
        #featuresdfNew.to_pickle(subfolder+'pickle/reliefF_'+str(noFeatures)+'_'+selectVA+'.pkl')
        
        indexes = pd.DataFrame(indexes, columns=['indexes'])
        indexes.to_pickle(subfolder+'indexes/reliefF_'+str(noFeatures)+'_'+selectVA+'_indexes.pkl')
        
    noFeatures -= 10


valence 5
['MEL spectrogram 66', 'MEL spectrogram 65', 'MEL spectrogram 60', 'MEL spectrogram 80', 'Chroma B']

arousal 5
['Tonnetz 5y', 'Tonnetz 5x', 'MEL spectrogram 44', 'Chroma B', 'MEL spectrogram 45']


In [15]:
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
from commonFunctions import featureTranslation

subfolder = 'Dataset/'
indexesFeatures = 5 # from 5 to 80 -> 90 is special for all features
VA = ['valence', 'arousal']
selectVA = VA[0]
selectionType = "reliefF"


dictOfFeatId = None

while indexesFeatures < 90:
    selectedFeatIds = pd.read_pickle(subfolder+'indexes/'+selectionType+'_'+str(indexesFeatures)+'_'+selectVA+'_indexes.pkl')
    indexes = np.array(selectedFeatIds['indexes'].tolist()) # read indexes file
    
    
    if dictOfFeatId is None:
        dictOfFeatId = { i : 1 for i in indexes }
    else:
        for feat in indexes:
            if feat in dictOfFeatId:
                dictOfFeatId[feat] += 1
            else:
                 dictOfFeatId[feat] = 1
        
    if(indexesFeatures == 5):
        indexesFeatures+=5
    else:
        indexesFeatures+=10

#print(dictOfFeatId)

dictOfFeat = {}
for featId in dictOfFeatId:
    dictOfFeat[featureTranslation(featId)] = dictOfFeatId[featId]
    
print(dict(sorted(dictOfFeat.items(), key=lambda item: item[1], reverse=True)))






{'MEL spectrogram 66': 7, 'MEL spectrogram 53': 6, 'MEL spectrogram 55': 6, 'MEL spectrogram 65': 5, 'Chroma B': 5, 'MEL spectrogram 52': 5, 'Tonnetz 5x': 5, 'MEL spectrogram 74': 5, 'MEL spectrogram 78': 5, 'MEL spectrogram 98': 5, 'MEL spectrogram 5': 5, 'MEL spectrogram 62': 5, 'MEL spectrogram 60': 4, 'MEL spectrogram 80': 4, 'MEL spectrogram 47': 4, 'MEL spectrogram 45': 4, 'MEL spectrogram 57': 4, 'MEL spectrogram 59': 4, 'MEL spectrogram 56': 4, 'Chroma C#': 4, 'MEL spectrogram 79': 4, 'Chroma A#': 4, 'Chroma F#': 4, 'Chroma C': 4, 'Chroma E': 4, 'Spectral Contrast 3': 4, 'MEL spectrogram 6': 4, 'MEL spectrogram 87': 4, 'MEL spectrogram 42': 4, 'MEL spectrogram 20': 4, 'MEL spectrogram 69': 4, 'MEL spectrogram 21': 4, 'MEL spectrogram 83': 4, 'MEL spectrogram 12': 4, 'MEL spectrogram 68': 4, 'MEL spectrogram 73': 4, 'MEL spectrogram 58': 4, 'MEL spectrogram 44': 3, 'MEL spectrogram 50': 3, 'MEL spectrogram 51': 3, 'MEL spectrogram 89': 3, 'MEL spectrogram 81': 3, 'Chroma D#': 3,